# Computation Times for MRS Phantom Framwork
This notebook calculates the computation times for all processing steps in the 3D MRS data simulation framework.

## Imports

In [1]:
## Imports
# Standard libraries
import numpy as np
import os

# Own libraries
from preprocessing.preprocess_df import create_metab_df

from DigitalPhantom import DigitalPhantom

from simulation.basis import Basis
from simulation.signalModels import SignalModel
from simulation.simulation import simulate_spectra

from utils.auxillary import downsample_mrsi, save_nifti_mrsi, timer, save_hdf5_mrsi
from utils.visualizations import plot_metabolite_map, plot_mrsi

## Settings

In [2]:
# skeleton = 'MRiLab'
skeleton = 'BigBrainMR'

# resolution = 1.0
# target_resolution = (5.0, 5.0, 5.0)

resolution = 3.0
target_resolution = (6.0, 6.0, 6.0)

## Metabolite Database Filtering

In [3]:
# Calculate the time taken to create the dataframe
with timer():
    # Create metabolite dataframe
    metab_df, concs_df, t2_df = create_metab_df(
                labels=['Background', 'WM', 'GM', 'CSF'],   # Tissue labels to include
                groups=['Healthy', 'Control'],              # Group labels to include
                fraction_boundary=0.6,                      # Tissue fraction boundary for GM/WM labelling
                age_range=[18, 60],                         # Age range for studies to include
                tesla=3.0,                                  # Magnetic field strength to include               
                save=True,                                  # Save created dataframe to a csv file          
    )

LOADING MRS DATABASE...
Preprocess concentration dataframe...
Filtering on "['Healthy', 'Control']" group... 
          Before: Unique articles - 351, Entries - 9136 
          After : Unique articles - 291, Entries - 3812
        
Dropping entries where no information about GM/WM is present... 
          Before: Unique articles - 291, Entries - 3812 
          After : Unique articles - 100, Entries - 1412
        
Change percentages to fractions...
Assign tissue value based on GM/WM fractions (boundary: 0.6)...
Drop all entries that are not WM or GM... 
          Before: Unique articles - 100, Entries - 1412 
          After : Unique articles - 71, Entries - 847
        
Change NaN values to 0...
Select age range of 18-60 years... 
          Before: Unique articles - 71, Entries - 847 
          After : Unique articles - 54, Entries - 585
        
Remove all MM entries... 
          Before: Unique articles - 54, Entries - 585 
          After : Unique articles - 54, Entries - 585
    

## Generating Phantom

In [4]:
# Track the time taken to create the phantom
with timer():
    # Create a DigitalPhantom object
    phantom = DigitalPhantom(
        # General settings
        skeleton=skeleton,                         
        resolution=resolution,                                    
        path2phantom='./data/phantom/',            
        from_scratch=True,                     
        # Metabolite settings
        concs_std=0.05,                         
        # Lipid settings
        sigma_lipid_spread=3.5,                     
        # Gradient settings
        grad_metabs=['NAA'], 
        grad_settings=[0.5, 3, '+y']
        )

Initializing Digital Phantom...
Creating a new phantom from scratch...
Creating phantom data...
Creating phantom dictionary...


100%|██████████| 84/84 [00:02<00:00, 29.40it/s]


Populating phantom array...


100%|██████████| 185900/185900 [00:01<00:00, 133155.40it/s]


Saving phantom data...
Saving NIfTI files...


100%|██████████| 5/5 [00:00<00:00,  9.53it/s]


Phantom data saved successfully to "./data/phantom/BigBrainMR/3.0mm"
Phantom created and saved at "./data/phantom/BigBrainMR/3.0mm".
Duration: 11.1167 seconds


## Loading Phantom

In [5]:
# Track the time taken to create the phantom
with timer():
    # Create a DigitalPhantom object
    phantom = DigitalPhantom(
        # General settings
        skeleton=skeleton,                         
        resolution=resolution,                                    
        path2phantom='./data/phantom/',            
        from_scratch=False,                     
        # Metabolite settings
        concs_std=0.05,                         
        # Lipid settings
        sigma_lipid_spread=3.5,                     
        # Gradient settings
        grad_metabs=['NAA'], 
        grad_settings=[0.5, 3, '+y']
        )

Initializing Digital Phantom...
Phantom found at "./data/phantom/BigBrainMR/3.0mm". Loading phantom...
Phantom loaded!
Duration: 0.3719 seconds


## Spectral Simulation
Here, all slices will be simulated.

In [6]:
path2basis = './data/Basissets/3T/Philips_TE30_PRESS_pMMupdated.mat'

phantom_metabs = list(phantom.metab_mapping.keys())

basis = Basis(path2basis, fmt='osprey', bw=2000, points=2048, metabs=[], phantom_metabs=phantom_metabs)
signalModel = SignalModel(basis, gauss_broadening=15, mm_scale=1000, noise_std=1000, res_water_scale=5, lipid_scale=10, baseline_order=2, baseline_scale=400)

with timer():   
    ppm_axis, spectra = simulate_spectra(phantom, basis, sigModel=signalModel, metabs=[], slice_range=[], batch_size=1000)

print(f"\nSize of the simulated spectra: {spectra.shape}")

There are metabolites in the data that are not present in the basis: ['Glc', 'NAA_Asp']
Removing these metabolites from the data...
Simulating spectra for 61412 voxels...


100%|██████████| 62/62 [01:13<00:00,  1.19s/it]

Spectral simulation completed!
Duration: 73.8573 seconds

Size of the simulated spectra: (52, 65, 55, 2048)


## Downsampling MRSI + Saving NIfTI-MRS file

In [7]:
# Save temporary file
with timer():
    # Save HDF5 file
    save_dir = '/Volumes/Samsung_T7/data/Digital_Brain_Phantom'
    os.makedirs(save_dir, exist_ok=True)

    save_name = 'mrsi_simulated_DEMO'
    file_name = save_hdf5_mrsi(spectra, save_dir, save_name, phantom)

    # Downsample and save nifti-mrsi
    path2save = os.path.join(phantom.path2phantom, 'mrsi')
    os.makedirs(path2save, exist_ok=True)
    reduced_data, new_affine = downsample_mrsi(file_name, phantom.affine, target_resolution=target_resolution)
    save_nifti_mrsi(reduced_data, basis, new_affine, etime=35, rtime=1.0, path2save=path2save, save_name='DEMO_RUN')

Saving MRSI data as HDF5 file to /Volumes/Samsung_T7/data/Digital_Brain_Phantom/BigBrainMR/3.0mm/mrsi_simulated_DEMO.h5...
Saving HDF5 file done!
Starting downsampling...
Padding MRSI data with [(0, 0), (0, 1), (0, 1), (0, 0)]...
Downsampling MRSI data from (3.0, 3.0, 3.0) mm to (6.0, 6.0, 6.0) mm...
[########################################] | 100% Completed | 7.32 sms
Downsampling completed!
Saving mrs-nifti file...
Done!
Duration: 15.9996 seconds
